Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
# import libraries
import pandas as pd 

In [5]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-01-24 03:20:06--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.171.13
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.171.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’

      instacart_onl  23%[===>                ]  46.09M  43.5MB/s               ^C


In [7]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [8]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [9]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [10]:
# view the different data frames
# aisles
# aisles contain aisles unique id number and name of isles
aisles = pd.read_csv('aisles.csv')
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [11]:
# departments
# departments contains unique department id's and name of deparments
departments = pd.read_csv('departments.csv')
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [12]:
# order_products__prior;
# contains order id, product id, order a product was added to the cart, 
# and if the product has been reordered
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [13]:
# order_products__train
# order id, product id, order product was added to cart, 
# and if product has been reordered again
order_products__train = pd.read_csv('order_products__train.csv')
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [14]:
# orders
# contains order id, user's id, data set it was taken from, which order this is for the user
# day of week order was mad, hour of the day order was made, days since previous order
orders = pd.read_csv('orders.csv')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [15]:
# products 
# product id, name of product, aisle product was from, and the department
products = pd.read_csv('products.csv')
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [0]:
# we're going to need to data frames with the following columns:
# product name, product id, and order id

# the data frames wth these: 
# product name(products)
# product_id(products, train, prior)
# order_id (orders, train, prior)

In [17]:
# concat the train and prior data frames 
order_products = pd.concat([order_products__prior, order_products__train], axis = 0)

order_products = order_products.reset_index()
print (order_products.shape)

order_products.head(10)

(33819106, 5)


,index,order_id,product_id,add_to_cart_order,reordered
0,0,2,33120,1,1
1,1,2,28985,2,1
2,2,2,9327,3,0
3,3,2,45918,4,1
4,4,2,30035,5,0
5,5,2,17794,6,1
6,6,2,40141,7,1
7,7,2,1819,8,1
8,8,2,43668,9,0
9,9,3,33754,1,1


In [0]:
# merge orders data frame to order_products
# but first view both

In [19]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [20]:
order_products.head()

,index,order_id,product_id,add_to_cart_order,reordered
0,0,2,33120,1,1
1,1,2,28985,2,1
2,2,2,9327,3,0
3,3,2,45918,4,1
4,4,2,30035,5,0


In [0]:
# slim down both products and order_products to the specific order_id's we'll need


In [0]:
# slim down products using a condition that selects out top 10 products
# in our product category
condition = products['product_name'].isin(['Banana', 'Bag of Organic Bananas',
                                           'Organic Strawberries', 
                                           'Organic Baby Spinach', 
                                           'Organic Hass Avocado', 
                                           'Organic Avocado',
                                           'Large Lemon',
                                           'Strawberries',
                                           'Limes',
                                           'Organic Whole Milk'])

In [22]:
# create a new dataframe with only the top 10 products
products_subset = products[condition]

# print the shape of the subset to make sure we have the right number of products
print(products_subset.shape)

# return the first five rows of that subset
products_subset.head(11)

(10, 4)


,product_id,product_name,aisle_id,department_id
13175,13176,Bag of Organic Bananas,24,4
16796,16797,Strawberries,24,4
21136,21137,Organic Strawberries,24,4
21902,21903,Organic Baby Spinach,123,4
24851,24852,Banana,24,4
26208,26209,Limes,24,4
27844,27845,Organic Whole Milk,84,16
47208,47209,Organic Hass Avocado,24,4
47625,47626,Large Lemon,24,4
47765,47766,Organic Avocado,24,4


In [0]:
# filter the order_products dateframe to only include the 10 items we're concerned with
condition2 = order_products['product_id'].isin([13176, 116797, 21137, 21903, 24852, 
                                                26209, 27845, 47209, 47626, 47765])


In [24]:
# create a new data frame for subset from order_products of
# the 10 products id's we're interested in
order_products_subset = order_products[condition2]

print(order_products_subset.shape)
order_products_subset.head(20) 

(2084759, 5)


,index,order_id,product_id,add_to_cart_order,reordered
12,12,3,21903,4,1
30,30,5,13176,1,1
51,51,5,47209,22,0
77,77,10,24852,1,1
83,83,10,21137,7,1
126,126,14,27845,2,1
180,180,20,24852,6,0
190,190,22,24852,3,1
234,234,26,24852,2,1
238,238,26,21903,6,0


In [25]:
# merge product_subset and order_products_subset
topten = pd.merge(order_products_subset, 
                  products_subset, 
                  on='product_id', 
                  how='inner')
print(topten.shape)
topten.head(10)

(2084645, 8)


,index,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,12,3,21903,4,1,Organic Baby Spinach,123,4
1,238,26,21903,6,0,Organic Baby Spinach,123,4
2,294,31,21903,3,1,Organic Baby Spinach,123,4
3,331,39,21903,4,0,Organic Baby Spinach,123,4
4,469,56,21903,8,1,Organic Baby Spinach,123,4
5,472,57,21903,1,0,Organic Baby Spinach,123,4
6,605,73,21903,4,1,Organic Baby Spinach,123,4
7,793,92,21903,2,1,Organic Baby Spinach,123,4
8,1016,113,21903,4,0,Organic Baby Spinach,123,4
9,1259,135,21903,5,1,Organic Baby Spinach,123,4


In [26]:
# drop the columns we don't need from out newly create topten data frame
topten = topten.drop(['reordered', 'order_id', 'add_to_cart_order', 'aisle_id',
                      'department_id'], axis=1)
topten.head(10)

,index,product_id,product_name
0,12,21903,Organic Baby Spinach
1,238,21903,Organic Baby Spinach
2,294,21903,Organic Baby Spinach
3,331,21903,Organic Baby Spinach
4,469,21903,Organic Baby Spinach
5,472,21903,Organic Baby Spinach
6,605,21903,Organic Baby Spinach
7,793,21903,Organic Baby Spinach
8,1016,21903,Organic Baby Spinach
9,1259,21903,Organic Baby Spinach


In [27]:
# reorder the columns
topten['product_name'].value_counts(10)

Banana                    0.235671
Bag of Organic Bananas    0.189447
Organic Strawberries      0.132194
Organic Baby Spinach      0.120742
Organic Hass Avocado      0.105954
Large Lemon               0.077132
Limes                     0.070353
Organic Whole Milk        0.068507
Name: product_name, dtype: float64

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [32]:
table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [33]:
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [31]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [34]:
# table2 -> tidy
# first reset the index
table2 = table2.reset_index()
table2.head()

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [39]:
# then use .melt() to convert into tidey
# identify the unique identifiers for each row
# where is the date that I want to be in my single "tidy" column
tidy = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])
tidy

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [42]:
# rename the columns
tidy = tidy.rename(columns={
    'index':'trt',
    'variable': 'name',
    'value': 'result'
})
tidy

,trt,name,result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [44]:
# ??????????????

#tidy -> table2
# we'll need to use the pivot_table function
# index = unique identifiers
# values = where are the numbers at -  what goes in the middle of the data frame
# columns = what do you want tp differentiate the columns in wide format
wide = tidy.pivot_table(index='trt', columns='name', values='result')
wide

name,Jane Doe,John Smith,Mary Johnson
trt,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns

In [88]:
flights = sns.load_dataset('flights')
flights.head(50)

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121
5,1949,June,135
6,1949,July,148
7,1949,August,148
8,1949,September,136
9,1949,October,119


In [87]:
# Flights Pivot Table
wide2 = flights.pivot_table(index='year', columns='month', values='passengers')
wide2

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####